In [46]:
"""
Chapter 10. Network Models. Max Flow.
Anderson, David R., Dennis J. Sweeney,
Thomas A. Williams,
Jeffrey D. Camm y Kipp Martin
Quantitative Methods for Business, 11a. Ed
"""
from itertools import product
import math

import pulp as pl


In [47]:
nodes = range(7)

# adjacency_matrix
a = [
    [0, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0],
 ]

capacity = [
    [0, 5, 6, 5, 0, 0, 0],
    [0, 0, 2, 0, 3, 0, 0],
    [0, 2, 0, 3, 3, 7, 0],
    [0, 0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 0, 1, 8],
    [0, 0, 0, 0, 1, 0, 7],
    [10_000, 0, 0, 0, 0, 0, 0],
    ]

load = {
    0: 0, 
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
}

In [48]:

capacity = pl.makeDict((nodes, nodes), capacity, 1_000)


# problem
problem = pl.LpProblem('flowmax', pl.LpMaximize)


# variables
flow = pl.LpVariable.dicts('flow', (nodes, nodes), 0, None, pl.LpContinuous)

# objective function
problem += flow[6][0], 'max_flow_network'


# Transhipment Constraints
for k in nodes:
    problem += pl.lpSum(a[k][out] * flow[k][out] for out in nodes) - pl.lpSum(a[enter][k] * flow[enter][k] for enter in nodes) == load[k], f'flow_to_node_{k}'

# capacity constraints
for i in nodes:
    for j in nodes:
        problem += (flow[i][j] <= capacity[i][j])

In [49]:

solver = pl.get_solver('GUROBI') 
problem.solve(solver)

# The status of the solution is printed to the screen
print("Status:", pl.LpStatus[problem.status])


# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    if v.varValue > 1e-10:
        print(f'{v.name}: {v.varValue:,.2f}')

print(f'Total Vehicles = {pl.value(problem.objective) * 1000:,}')


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 56 rows, 49 columns and 79 nonzeros
Model fingerprint: 0xe95c149a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 50 rows and 37 columns
Presolve time: 0.01s
Presolved: 6 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4008000e+01   9.008000e+00   0.000000e+00      0s
       3    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  1.400000000e+01
Gurobi status= 2
Status: Optimal
flow_0_1: 5.00
flow_0_2: 4.00
flow_0_3: 5.00
flow_1_2: 2.00
flow_1_4: 3.00
flow_2_4: 3.00
flow_2_5: 3.00
flow_3_5: 5.00
flow_4_6: 7.00
flow_5_4: 1.00
flow_5_6: 7.00
flow_6_0: 14.00
Total Vehicles = 14,000.00
